<a href="https://colab.research.google.com/github/helmana/DLeeg/blob/master/2ch%20-%2010s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mne

    100% |████████████████████████████████| 6.2MB 5.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/e4/be/8b/6b737d272fb91d42e7ac5d61cc93227df7d71944e0e0eb15dd
Successfully built mne


In [2]:

import matplotlib

from matplotlib import pyplot as plt
import numpy as np



import random
import mne
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
from mne import io
from mne import viz
#from mne.datasets import testing
from mne import Epochs, io, pick_types
from mne.event import define_target_events
from mne.time_frequency import psd_welch
print(__doc__)

Automatically created module for IPython interactive environment


In [0]:
#with np.errstate(divide='ignore'):
   # np.float64(1.0) / 0.0

In [3]:
subject_number=10


# load dataset in array
list_raw_fnames = [[0]*2]*subject_number
for x in range(subject_number):
    list_raw_fnames[x] = mne.datasets.eegbci.load_data(x+1,[1,2])

list_rawdata = np.zeros((subject_number,2), dtype='object')

for i in range(subject_number):
    for j in range(2):
        list_rawdata[i][j] = mne.io.read_raw_edf(list_raw_fnames[i][j], preload=True)
print( list_rawdata[:][:])

Using default location ~/mne_data for EEGBCI...
Creating ~/mne_data
[........................................] 100.00% (  1.2 MB,  11.2 MB/s) \   
Do you want to set the path:
    /root/mne_data
as the default EEGBCI dataset path in the mne-python config [y]/n? n
Using default location ~/mne_data for EEGBCI...
[........................................] 100.00% (  1.2 MB,  14.4 MB/s) \   
Do you want to set the path:
    /root/mne_data
as the default EEGBCI dataset path in the mne-python config [y]/n? y
Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
[........................................] 100.00% (  1.2 MB,  20.1 MB/s) |   
[........................................] 100.00% (  1.2 MB,  17.6 MB/s) |   
[........................................] 100.00% (  1.2 MB,  21.8 MB/s) |   
[........................................] 100.00% (  1.2 MB,  12.7 MB/s) \   
[........................................] 100.00% (  1.2 MB,  16.1 MB/s) |   
[.............

In [4]:
import math


task_number=1
task_time = 60
sampel_number_per_sec =  160 # sampel rate
total_sampel_number =  sampel_number_per_sec *task_time # 60*160
sample_shift = 5 #step len
window_len= 10

# img for each one
subject_img_number = math.floor((total_sampel_number - sampel_number_per_sec) / sample_shift) +1 - window_len +1
test_start_index=subject_img_number - math.floor(subject_img_number/4) 

train_number =  test_start_index 
test_number = subject_img_number - test_start_index
train_number

1410

In [5]:
ch_number = 2

train_img = np.zeros((train_number*subject_number, ch_number, window_len + 1, 160), dtype = float)
train_label =[]


test_img = np.zeros((test_number*subject_number, ch_number, window_len, 160), dtype = float)
test_label =[]


train_img.shape

(14100, 2, 11, 160)

In [0]:

def normalize_channel_data(ch , i, ch_min, ch_max):
  ch = ((ch - ch_min[i]) / (ch_max[i] - ch_min[i] ) )
  return ch

In [0]:
for s in range(subject_number):
    rawdataChannels_t, times_t=rawdataChannels, times =list_rawdata[s][0][:64,:9600]
    ch_max =[]
    ch_min =[]
    for p in range(len(rawdataChannels_t)):
        ch_max = np.append(ch_max, max(rawdataChannels_t[p])) # max for each cannel
        ch_min = np.append(ch_min, min(rawdataChannels_t[p])) # min for each cannel

    for j in range (subject_img_number):
        for i in range(ch_number):
            for z in range (window_len):
                rawdataChannels, times=rawdataChannels, times =list_rawdata[s][0][i,0+(j+z)*sample_shift:sampel_number_per_sec+(j+z)*sample_shift]

                      # normalize method
                rawdataChannels = normalize_channel_data(rawdataChannels, i, ch_min, ch_max)
                if j <test_start_index :
                    train_img[s*train_number + j][i][z] = rawdataChannels
                    if z==window_len-1 :
                      train_img[s*train_number + j][i][z+1] = s
                      
                      
                else:
                    test_img[s*test_number + j - test_start_index][i][z] = rawdataChannels
        if j >=test_start_index :
          test_label = np.append(test_label, (s))

           

In [0]:
#shaffle train img array
train_img_shuffle = train_img

np.random.shuffle(train_img_shuffle)

In [0]:
train_img_2 = np.zeros((train_number*subject_number, ch_number, window_len, 160), dtype = float)


In [0]:
train_img_2.shape

(7050, 5, 10, 160)

In [0]:
# img label 

train_img_shuffle_len =len(train_img_shuffle)
for i in range(train_img_shuffle_len):
    train_label = np.append(train_label, (train_img_shuffle[i][0][window_len][0] ))
    for j in range (ch_number):
      train_img_2[i][j] = np.delete(train_img_shuffle[i][j], window_len, axis=0)

In [11]:
import keras
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
train_label = to_categorical(train_label, subject_number)
test_label = to_categorical(test_label, subject_number)

In [13]:
# valid & train
x_train =train_img_2[:8000]
y_train =train_label[:8000]

x_valid =train_img_2[8000:]
y_valid =train_label[8000:]

x_train.shape

(8000, 2, 10, 160)

In [14]:
from keras import layers
from keras import models
from keras import regularizers


model = models.Sequential()
model.add(layers.Conv2D(64, (3,3), activation = 'relu', input_shape = (ch_number,window_len,160), data_format= "channels_first" ))
print(model.output.shape)
model.add(layers.MaxPooling2D((2,2)))
print(model.output.shape)
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
print(model.output.shape)
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
print(model.output.shape)
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dropout(0.5))
print(model.output.shape)
model.add(layers.Dense(subject_number, activation = 'softmax'))
print(model.output.shape)

Instructions for updating:
Colocations handled automatically by placer.
(?, 64, 8, 158)
(?, 32, 4, 158)
(?, 30, 2, 128)
(?, ?)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
(?, 512)
(?, 10)


In [0]:

from keras import optimizers

model.compile(loss= 'categorical_crossentropy',
              optimizer= optimizers.RMSprop(lr= 1e-4),
              metrics = ['acc'])

In [16]:
history = model.fit(
        x_train,
        y_train,
        epochs = 15,
        batch_size = 10,
        validation_data = (x_valid, y_valid)
)

Instructions for updating:
Use tf.cast instead.
Train on 8000 samples, validate on 6100 samples
Epoch 1/15
8000/8000 [==============================] - 55s 7ms/step - loss: 1.7326 - acc: 0.3679 - val_loss: 1.1331 - val_acc: 0.6608
Epoch 2/15
8000/8000 [==============================] - 55s 7ms/step - loss: 1.0113 - acc: 0.6303 - val_loss: 0.7740 - val_acc: 0.7152
Epoch 3/15
8000/8000 [==============================] - 54s 7ms/step - loss: 0.7623 - acc: 0.7186 - val_loss: 0.6067 - val_acc: 0.7595
Epoch 4/15
8000/8000 [==============================] - 54s 7ms/step - loss: 0.5994 - acc: 0.7869 - val_loss: 0.4481 - val_acc: 0.8669
Epoch 5/15
8000/8000 [==============================] - 56s 7ms/step - loss: 0.4470 - acc: 0.8505 - val_loss: 0.3446 - val_acc: 0.8866
Epoch 6/15
8000/8000 [==============================] - 55s 7ms/step - loss: 0.3326 - acc: 0.8910 - val_loss: 0.3282 - val_acc: 0.8816
Epoch 7/15
8000/8000 [==============================] - 55s 7ms/step - loss: 0.2471 - acc: 0.9

In [17]:
model.evaluate(test_img, test_label)


4700/4700 [==============================] - 7s 1ms/step


[0.1348031418279795, 0.943404255319149]